In [6]:
from neural_language_model import TextDataset, create_dataloaders, ft_embedding

In [ ]:
# import zipfile
# path_to_zip_file = '/scratch/hmnshpl/anlp_data/glove.6B.zip'
# directory_to_extract_to = '/scratch/hmnshpl/anlp_data'
# zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
# zip_ref.extractall(directory_to_extract_to)
# zip_ref.close()

In [5]:
print(f'loading embedding', end='\r')
embedding_model = ft_embedding.load_glove_embeddings()
print(f'loaded embedding     ')

file_path = '/scratch/hmnshpl/anlp_data/Auguste_Maquet.txt'  # Example path to dataset file
embedding_dim = 300  # Make sure it matches the dimensionality of your pre-trained embeddings

# Create the dataset instance
dataset = TextDataset(file_path, embedding_model, embedding_dim=embedding_dim)

# Create DataLoader
batch_size = 32  # Example batch size
train_loader = create_dataloaders(dataset, batch_size=batch_size)

# Example of iterating through the DataLoader
for batch in train_loader:
    contexts, targets = batch
    print("Contexts Shape:", contexts.shape)  # Should be (batch_size, context_size, embedding_dim)
    print("Targets Shape:", targets.shape)    # Should be (batch_size,)
    break  # Remove this line to go through the whole dataset


loaded embedding     
Contexts Shape: torch.Size([32, 5, 300])
Targets Shape: torch.Size([32])
